In [130]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# import findspark
# findspark.init()
if sc: sc.stop()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, ArrayType, StringType

sc = SparkContext()
spark = SparkSession(sc)

In [142]:
schema = StructType([StructField('temps',ArrayType(StructType( \
              [StructField('current',ArrayType(StructType \
                  ([StructField('temp',StringType(),True)]),True),True), \
               StructField('hist',ArrayType(StructType \
                  ([StructField('temp',StringType(),True)]),True), \
         True)]),True),True),
         StructField('uuid',StringType(),True)])

In [278]:
B = [Row(uuid='101', temps=[Row(current=[Row(temp=None)], hist=[Row(temp='100')])])]
A = [Row(uuid='101', temps=[Row(current=[Row(temp='101')], hist=[Row(temp=None)])])]

dfA = sc.parallelize(A).toDF(schema)
dfA.registerTempTable('dfA')
dfB = sc.parallelize(B).toDF(schema)
dfB.registerTempTable('dfB')
dfA.collect()
dfB.collect()

[Row(temps=[Row(current=[Row(temp='101')], hist=[Row(temp=None)])], uuid='101')]

[Row(temps=[Row(current=[Row(temp=None)], hist=[Row(temp='100')])], uuid='101')]

In [267]:
# dfA.printSchema()

dfC = dfA \
    .select(F.explode('temps').alias('tmp')) \
    .select('tmp.*') \
    .show()

+-------+----+
|current|hist|
+-------+----+
|[[101]]|[[]]|
+-------+----+



In [271]:
dfA.select('temps.current.temp').show()

AnalysisException: "cannot resolve '`temps`.`current`['temp']' due to data type mismatch: argument 2 requires integral type, however, ''temp'' is of string type.;;\n'Project [temps#547.current[temp] AS temp#846]\n+- AnalysisBarrier\n      +- LogicalRDD [temps#547, uuid#548], false\n"

In [248]:
from pyspark.sql import functions as F

dfA = dfA.withColumn('temps_new', \
                    F.when(dfA.select(F.explode('temps').alias('tmp'))).select('tmp.current)'==None, '999999').
                    otherwise('sdfsdfds'))

TypeError: when() missing 1 required positional argument: 'value'

In [272]:
from pyspark.sql import functions as F

spark_df = dfA.join(dfB, 'uuid', 'inner').withColumn('temps',\
           F.when((dfB.temps.current == None) , dfA.temps.current)\
           .otherwise(dfB.temps))


# spark_df = table_1.join(table_2, 'uuid', 'inner').withColumn('list_expire_value',
#            F.when((table_2.list_expire_value > 5) | (table_2.list_date < 6), table_1.listed_1)
#            .otherwise(table_2.list_date)).drop(table_1.listed_1)

AnalysisException: "cannot resolve 'CASE WHEN (`temps`.`current` = NULL) THEN `temps`.`current` ELSE `temps` END' due to data type mismatch: THEN and ELSE expressions should all be same type or coercible to a common type;;\n'Project [uuid#548, CASE WHEN (temps#551.current = cast(null as array<array<struct<temp:string>>>)) THEN temps#547.current ELSE temps#551 END AS temps#850, CASE WHEN (temps#551.current = cast(null as array<array<struct<temp:string>>>)) THEN temps#547.current ELSE temps#551 END AS temps#851]\n+- AnalysisBarrier\n      +- Project [uuid#548, temps#547, temps#551]\n         +- Join Inner, (uuid#548 = uuid#552)\n            :- LogicalRDD [temps#547, uuid#548], false\n            +- LogicalRDD [temps#551, uuid#552], false\n"

In [280]:
req="""
    SELECT struct(*) from dfA
   """

spark.sql(req).show()


+--------------------------------------+
|named_struct(temps, temps, uuid, uuid)|
+--------------------------------------+
|                  [[[[[101]], [[]]]...|
+--------------------------------------+

